In [2]:
import pandas as pd
import pysrt
import re
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
def sub_preprocessing(filename):
    # Читаем файл субтитров
    try:
        subs = pysrt.open(filename, encoding='iso-8859-1')
        print('Success opening')
    except Exception as ex:
        print('Unknown type of file')
        return


    if subs[-1].index - subs[-2].index < 2:
            film_end = subs[-1].end.hours*3600 + subs[-1].end.minutes*60 + subs[-1].end.seconds # Время окончания фильма
    else:
            film_end = subs[-2].end.hours*3600 + subs[-2].end.minutes*60 + subs[-2].end.seconds
        
    
    film_start, film_end = min(film_start, film_end), max(film_start, film_end)
    film_lenght = film_end - film_start # Продолжительность фильма
    # Далее почистим текст
    text = re.sub(r'\<.*?\>', '', subs.text) # Удалим <> и все что в них
    text = re.sub(r'\n', '', text) # Удалим знаки переноса строки
    text = re.sub(r'\(.*?\)', '', text) # Удалим () и все что в них
    text = re.sub('\[.*?\]', '', text)  # Удаляем то что в скобках []
    text = re.sub('[A-Z]+?:', '', text) # Удаляем слова написанные заглавными буквами с двоеточием(это имена тех кто говорит)
    text = re.sub('\.+?:', '\.', text)  # Заменяем троеточия на одну точку
    text = text.lower()
    text = re.sub('[^a-z\.\!\?]', ' ', text) # Удаляем всё что не буквы и не .?!
    text = re.sub(' +', ' ', text) # Удаляем " +"
    
    num_sentence = len(re.split('[\.\?\!]', text)) # Количетсво предложений
    
    morphs = [_[1] for _ in nltk.pos_tag(re.sub('\n', ' ', text).lower().split(' '))] # Морфологический разбор 
    gerund = morphs.count('VBG')  # Герундий
    text_no_preproc = text
    text = re.sub('[^a-z]', ' ', text) # Удалим всё что не буквы
    text_len = len(text) # Количество символов
    sub_per_second = text_len / film_lenght # Букв в секунду
    
    for stop_word in sw.words('english'):
        text = re.sub(f' {stop_word} ', ' ', text) # Удалим стоп-слова
        
    text = re.sub(' [a-z] ', ' ', text) # Удалим однобуквенные слова
    
    words = text.split(' ')
    words_unique = []
    for word in words:
        if word not in words_unique:
            words_unique.append(word) # Список уникальных слов
    
    sub_per_word = (text_len - len(words)) / len(words)
        
     
    difficulty = {'A2':0,
                  'B1':0,
                  'B2':0,
                  'C1':0} # Расчитаем количество слов разной сложности в фильме

    for word in words_unique:
        match = Oxford[Oxford['words'] == word]['level'].values
        if len(match) > 0:
            for lvl in match:
                difficulty[lvl] += 1

    wnl = WordNetLemmatizer()
    text = ' '.join([wnl.lemmatize(word, wordnet.VERB) for word in text.split(' ')]) # Лемматизация
    phrases_lenght = 0  # Посчитаем продолжительность всех фраз
    for sub in subs:   
        phrase_start = sub.start.hours*3600 + sub.start.minutes*60 + sub.start.seconds # Время начала фразы
        phrase_end = sub.end.hours*3600 + sub.end.minutes*60 + sub.end.seconds # Время окончания фразы
        phrases_lenght += max(0, phrase_end - phrase_start) # Продолжительность фразы
        if phrases_lenght != 0:
            sub_per_second_frases = text_len / phrases_lenght

    return pd.DataFrame({'text_len': [len(text)], # Возвращаем результаты
            'sub_per_second': [sub_per_second], 
            'phrases_lenght': [phrases_lenght],
            'sub_per_second_frases': [sub_per_second_frases],
            'sub_per_word': [sub_per_word],
            'num_sentence': [max(num_sentence, len(subs))],
            'word_persentence': [len(words) / max(num_sentence, len(subs))],
            'A2': [difficulty['A2']/len(words_unique)],
            'B1': [difficulty['B1']/len(words_unique)],
            'B2': [difficulty['B2']/len(words_unique)],
            'C1': [difficulty['C1']/len(words_unique)],
            'B1+': [1 - (difficulty['A2'] + difficulty['B1'] + difficulty['B2'] + difficulty['C1'])/len(words_unique)],
            'words_unique_count': [len(words_unique)],
            'words_unique_per_second': [len(words_unique)/film_lenght],
            'words_unique_part': [len(words_unique)/len(words)],
            'gerund': [gerund],
            'gerund_per_sentence': [gerund/num_sentence]})